In [1]:
from ...server.automl.neural_network.neural_network import NeuralNetwork
from ...server.automl.neural_network.fc_layer import FCLayer
from ...server.automl.neural_network.activation_layer import ActivationLayer
from ...server.automl.neural_network.activation_functions import tanh, tanh_derivative, relu, relu_derivative
from ...server.automl.neural_network.loss_functions import mse, mse_derivative
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
X, Y = load_diabetes(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(442, 10) (442,)
(353, 10) (89, 10) (353,) (89,)


In [3]:
def my_func(config, reporter):  # add the reporter parameter
      # report the configuration
    network = NeuralNetwork()
    network.add(FCLayer(config['input_dim'], config['hidden_dim']))
    network.add(ActivationLayer(relu, relu_derivative))
    network.add(FCLayer(config['hidden_dim'], 2))
    network.add(ActivationLayer(relu, relu_derivative))
    network.add(FCLayer(2, config['output_dim']))
    network.add(ActivationLayer(relu, relu_derivative))
    network.use(mse, mse_derivative)
    network.fit(X_train, y_train, epochs=90000, learning_rate=config["alpha"])
    y_pred = network.predict(X_train)
    reporter(config, mean_loss = mean_squared_error(y_test, y_pred))

In [4]:
algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
algo = ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()
analysis = tune.run(
        my_func,
        name="my_exp",
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=10,
        config={
            "steps": 100,
            "input_dim": X_train.shape[1],
            "hidden_dim": 5,
            "output_dim": 1,
            "alpha": tune.uniform(0.0001, 10),
        },
    )

print("Best hyperparameters found were: ", analysis.best_config)

C:\Users\Public\anaconda3\lib\site-packages\ray\_private\services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2022-04-14 05:33:50,097	WARNING bayesopt.py:397 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.


Trial name,status,loc,alpha
my_func_b33f7da6,PENDING,,3.74546


2022-04-14 05:33:50,521	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
2022-04-14 05:33:52,278	WARNING worker.py:1215 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID 63c321026544a2981c4fc55201000000. Traceback:
Traceback (most recent call last):
  File "C:\Users\Public\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 503, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
ModuleNotFoundError: No module named 'activation_layer'

(pid=13076) 2022-04-14 05:33:52,268	ERROR function_manager.py:505 -- Failed to load actor class ImplicitFunc.
(pid=13076) Traceback (most recent call last):
(pid=13076)   File "C:\Users\Public\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 503, in _load_actor_class_from_gcs
(pid=13076)     actor_class = pickle.loads(pickled_class)
(pid=13076) ModuleNotFoundError: No module named 'activation_layer'
(pid=13076) 2022-04-14 05:33:52,274	ERROR worker.py:428 -- Exception rai

Result for my_func_b36193d5:
  {}
  
Result for my_func_b3760bbd:
  {}
  


2022-04-14 05:33:52,772	WARNING worker.py:1215 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID 494210fedae1421d29b0296001000000. Traceback:
Traceback (most recent call last):
  File "C:\Users\Public\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 503, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
ModuleNotFoundError: No module named 'activation_layer'

(pid=13796) 2022-04-14 05:33:52,755	ERROR function_manager.py:505 -- Failed to load actor class ImplicitFunc.
(pid=13796) Traceback (most recent call last):
(pid=13796)   File "C:\Users\Public\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 503, in _load_actor_class_from_gcs
(pid=13796)     actor_class = pickle.loads(pickled_class)
(pid=13796) ModuleNotFoundError: No module named 'activation_layer'
(pid=13796) 2022-04-14 05:33:52,760	ERROR worker.py:428 -- Exception raised in creation task: The actor died because of an error raised in its creation task, r

...

Trial name,status,loc,alpha
my_func_b68a35dc,RUNNING,,7.08075
my_func_b33f7da6,ERROR,,3.74546
my_func_b36193d5,ERROR,,9.50715
my_func_b3760bbd,ERROR,,7.31997
my_func_b38eacd6,ERROR,,5.98662
my_func_b488dc27,ERROR,,1.56027
my_func_b4a26d2b,ERROR,,1.56003
my_func_b4ce1a31,ERROR,,0.58093
my_func_b5aa0c5f,ERROR,,8.66177
my_func_b64f510a,ERROR,,6.01119


2022-04-14 05:34:02,729	WARNING worker.py:1215 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID 820538cfdbc203c69ba4af7201000000. Traceback:
Traceback (most recent call last):
  File "C:\Users\Public\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 503, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
ModuleNotFoundError: No module named 'activation_layer'

(pid=15636) 2022-04-14 05:34:02,690	ERROR function_manager.py:505 -- Failed to load actor class ImplicitFunc.
(pid=15636) Traceback (most recent call last):
(pid=15636)   File "C:\Users\Public\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 503, in _load_actor_class_from_gcs
(pid=15636)     actor_class = pickle.loads(pickled_class)
(pid=15636) ModuleNotFoundError: No module named 'activation_layer'
(pid=15636) 2022-04-14 05:34:02,695	ERROR worker.py:428 -- Exception raised in creation task: The actor died because of an error raised in its creation task, r

Result for my_func_b68a35dc:
  {}
  


Trial name,status,loc,alpha
my_func_b33f7da6,ERROR,,3.74546
my_func_b36193d5,ERROR,,9.50715
my_func_b3760bbd,ERROR,,7.31997
my_func_b38eacd6,ERROR,,5.98662
my_func_b488dc27,ERROR,,1.56027
my_func_b4a26d2b,ERROR,,1.56003
my_func_b4ce1a31,ERROR,,0.58093
my_func_b5aa0c5f,ERROR,,8.66177
my_func_b64f510a,ERROR,,6.01119
my_func_b68a35dc,ERROR,,7.08075


TuneError: ('Trials did not complete', [my_func_b33f7da6, my_func_b36193d5, my_func_b3760bbd, my_func_b38eacd6, my_func_b488dc27, my_func_b4a26d2b, my_func_b4ce1a31, my_func_b5aa0c5f, my_func_b64f510a, my_func_b68a35dc])